In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential,layers
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer 
from collections import Counter
np.random.seed(4864)

In [ ]:
def cleanse(row):
    row_clean = row.replace("-", " ")
    row_clean = re.sub("[\n]", " ",row_clean)
    row_clean = re.sub("[.!?/\()-,:]", "",row_clean)
    row_clean = row_clean.lower()
    return row_clean

def clean_df(df):
    text = []
    target = []
    for row in df.iterrows():
            row_clean = cleanse(row[1][1])
            text.append(row_clean)
    out = pd.DataFrame({'text':text,"target":df.out})
    return out
#Create data partitions
df = pd.read_csv(f'/content/modeling_data.csv', encoding='latin-1')
df = df.dropna()
df = clean_df(df)
max_length = max(df.text.apply(len))
#Reduce amount of 0 targets as model was simply predicting 0 due to high proportion
# description_indexes = df.index[df.target == 0 ]
# df = df.drop(description_indexes[:int(len(description_indexes) * 0.5)]) 
#Shuffle dataframe
# df = df.sample(frac=1)
X_train, X_test, y_train, y_test=train_test_split(df.text, df.target, test_size=0.1)

In [ ]:
def counter_word(text):
  count = Counter()
  for i in text.values:
    for word in i.split():
      count[word] += 1 
  return count

In [ ]:
count = counter_word(X_train)
num_words = 30


In [ ]:
tokenizer = Tokenizer(num_words=num_words) 
tokenizer.fit_on_texts(texts = X_train)
train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])
decode(train_seq[58])   

'skills will you the to'

In [ ]:
def Tokenize_data(data,tokinizer):
  seq = tokenizer.texts_to_sequences(data)
  padded = pad_sequences(seq,padding = 'post',truncating='post')
  return padded

In [ ]:
t = ["require to know R and Python"]
t = Tokenize_data(t,tokenizer)

In [ ]:
train_padded = pad_sequences(train_seq,padding = 'post',truncating='post',maxlen= max_length)
test_padded = pad_sequences(test_seq,padding = 'post',truncating='post',maxlen= max_length)
words = len(tokenizer.word_counts)

In [ ]:
model = Sequential()
model.add(layers.Embedding(num_words, 128, input_length=max_length))
model.add(layers.LSTM(128,return_sequences =True))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1187, 128)         3840      
_________________________________________________________________
lstm_11 (LSTM)               (None, 1187, 128)         131584    
_________________________________________________________________
flatten_11 (Flatten)         (None, 151936)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 151937    
Total params: 287,361
Trainable params: 287,361
Non-trainable params: 0
_________________________________________________________________


In [ ]:
adam = tf.keras.optimizers.Adam(lr=.001, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=adam,loss= 'binary_crossentropy',metrics = ['accuracy','Recall'])

model.fit(x = train_padded, y= y_train,epochs=15,validation_split=.1)

Epoch 1/15


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


47/47 [==============================] - 11s 192ms/step - loss: 0.6291 - accuracy: 0.6740 - recall: 0.0170 - val_loss: 0.5735 - val_accuracy: 0.6867 - val_recall: 0.0000e+00
Epoch 2/15
47/47 [==============================] - 8s 177ms/step - loss: 0.6625 - accuracy: 0.6801 - recall: 0.4021 - val_loss: 0.4446 - val_accuracy: 0.8133 - val_recall: 0.4231
Epoch 3/15
47/47 [==============================] - 8s 177ms/step - loss: 0.4429 - accuracy: 0.8256 - recall: 0.5085 - val_loss: 0.3882 - val_accuracy: 0.8494 - val_recall: 0.5385
Epoch 4/15
47/47 [==============================] - 8s 177ms/step - loss: 0.3912 - accuracy: 0.8504 - recall: 0.6021 - val_loss: 0.3579 - val_accuracy: 0.8373 - val_recall: 0.5577
Epoch 5/15
47/47 [==============================] - 8s 177ms/step - loss: 0.3674 - accuracy: 0.8524 - recall: 0.6404 - val_loss: 0.3470 - val_accuracy: 0.8494 - val_recall: 0.5769
Epoch 6/15
47/47 [==============================] - 8s 177ms/step - loss: 0.3541 - accuracy: 0.8632 - reca

Now that we see validation metrics is being properly optimized lets train on all data

In [ ]:
model.evaluate(test_padded,y_test)

6/6 [==============================] - 1s 83ms/step - loss: 0.3973 - accuracy: 0.8270 - recall: 0.5484


[0.39734187722206116, 0.8270270228385925, 0.5483871102333069]

In [ ]:
def got_wrong(datax,datay,datax_pad,df):
  binray_preds = np.reshape(np.round(model.predict(datax_pad)),(len(datay)))
  true=np.array(datay)
  q = binray_preds != true
  match = datax[ binray_preds != true]
  out = df.loc[match.index ]
  return out

In [ ]:
incorrect = got_wrong(X_train,y_train,train_padded,df)
incorrect.tong_values.csv')

Model not very good